### Optimizer description: ZOO
**Name:** Zero Order Stochastic Coordinate Descent with Coordinat-wise (ADAM/Newton)<br>
**Class:** ZOOptim.ZOOptim <br>
**Paper:** *ZOO: Zeroth Order Optimization Based Black-box Attacks to
Deep Neural Networks without Training Substitute Models* (Chen, Zhang et al.) <br>


**Description:** <br>
The algorithm is based on the Carlini & Wagner white-box attck, which tries to find the adversarial example $x$ of the original image $x_0$ by solving the following problem:
$$min_x ||x-x_0||_2^2 + c\cdot f(x,t)$$
$$sub\: to\; x \in [0,1]^p$$

The term $f(x,t)$ is the loss function, defined as 
$$f(x,t)=max\{max_{i\neq t}[Z(x)]_i\, -\, [Z(x)]_t\;,\; -k\}$$

In the black-box framework we assume taht the logit layer representation is an internal state information of a DNN, therefore the loss function is modified in such a way that it only depends on the output F of a DNN and the desired class label t. The resulting loss function is the following:  

$f(x,t)=max\{max_{i\neq t}log[F(x)]_i\, -\, log[F(x)]_t\;,\; -k\}$ if the attack is targeted;  
$f(x)=max\{log[F(x)]_{t_0}\, -\, max_{t\neq t_0}log[F(x)]_i\;,\; -k\}$ if the attack is untargeted.<br>


An approximated gradient is computed as:

$$\hat{g_i}:=\frac{\partial f(x)}{\partial x_i}\approx \frac{f(x+he_i)-f(x-he_i)}{2h}\:,$$

where h is a small constant and $e_i$ is a standard basis vector with only the $i$-th component set to 1.<br>

For any $x \in \mathbb{R}^p$ , we need to evaluate the objective function $2p$ times to estimate gradients of all coordinates, which makes the approach too expensive in practice.  
With just one more function evaluation we can obtain an estimate of the Hessian as:

$$\hat{h_i}:=\frac{\partial^2 (x)}{\partial x^2_{ii}}\approx \frac{f(x+he_i)-2f(x)+f(x-he_i)}{h^2}\:.$$<br>

To overcome the computation cost stochastic coordinate descent is used, specifically it is shown that ADAM outperforms vanilla gradient descent and that in practice converges faster the Newton's method.

The algorithm is here reported: (Lo sistemerò fuori dal notebook)

Parameters: $\eta$, $M\in\mathbb{R}^p$, $T\in\mathbb{Z}^p$, $v\in\mathbb{R}^p$, $\beta_1$, $\beta_2$, $\epsilon$  
Until convergence:  
Randomly pick $B$ coordinates $i\in\{1,...,p\}$  
Estimate $\hat{g_B}$  
$T_B \leftarrow T_B+1$  
$M_B \leftarrow \beta_1 M_B+(1-\beta_1)\hat{g_B}$  
$v_B \leftarrow \beta_2 v_B+(1-\beta_2)\hat{g_B}^2$   
$\hat{M_B} = \frac{M_B}{(1-\beta_1^{T_B})}$   
$\hat{v_B} = \frac{v_B}{(1-\beta_2^{T_B})}$  
$\delta^* = -\eta \frac{\hat{M_B}}{\sqrt{\hat{v_B}}+\epsilon}$   
$x_B \leftarrow x_B + \delta^*$
    


**High Dimensional Extensions:** <br>
For network with large input size $p$ zero order methods can be quite slow. To avoid this problem we introduce a dimension reduction transformation $D(y)$, $D:\mathbb{R}^m \rightarrow \mathbb{R}^p$ with $m<p$ and replace it to $x-x_0$ in our minimization problem. In this way the dimension of an input remains unaltered but the dimension of the adversarial noise is reduced. 
For large images and difficult attacks a *hierarchical attack* scheme is proposed: a series of transformations $D1$, $D2$, $...$ with dimensions $m1$, $m2$, $...$ is used to gradually increase $m$ during the optimization process.

Another proposed technique is *importance sampling*, which consists in dividing the image in $8\times 8$ regions and assign sampling probabilities according to the pixels values change in that region.

The last techniques is *ADAM state reset* which is used to efficiently decrease the L2-distance after a valid attack has been found. The reason behind this idea is that after a valid adversarial image has been found and the hinge-loss function reaches 0, the optimizer still tries to reduce the probability of the original image instead to decrese the L2-distance.

**Args:**
 
    Name               Type                Description
    x:                 (torch.tensor)      The variable of our optimization problem. Should be a 3D tensor (img)
    c:                 (float)             Loss weight
    learning_rate:     (float)             Learning rate
    batch_size:        (int)               Coordinates we simultaneously optimize
    h:                 (float)             Gradient approximation accuracy O(h^2) 
    beta_1:            (float)             ADAM hyper-parameter
    beta_2:            (float)             ADAM hyper-parameter
    solver:            (str)               ADAM or Newton
    epsilon:           (float)             Avoid division by zero
    max_iterations:    (int)               The maximum number of steps
    stop_criterion     (boolean)           If true stop when the loss is 0
    verbose:           (int)               Display information. Default is 0
    tqdm_disable       (bool)              Disable the tqdm bar. Default is False
    additional_out     (bool)              Return also all the x. Default is False

                      
**Suggested values:** <br>
Chen and Zhang suggest using the following parameters:
* learning_rate = 0.001 
* batch_size = 128
* h = 0.0001
* beta_1 = 0.9
* beta_2 = 0.999
* solver = "adam"
* max_iterations = 10000

All this parameters are initialized by default.


**N.B** <br>
The original loss is equal to zero when more classes have the same probability. To avoid this issues and so to enforce that the desired class has the highest probability (targeted attack) or that the original class doesn't have the highest probability (untargeted attack) a small bias (1e-10) is added or subtracted accordingly.